In [1]:
import dagshub
import mlflow
mlflow.set_tracking_uri("https://dagshub.com/aryan0147/Capstone-Project-2.mlflow")

dagshub.init(repo_owner='aryan0147', repo_name='Capstone-Project-2', mlflow=True)

Accessing as aryan0147

Initialized MLflow to track repo "aryan0147/Capstone-Project-2"

Repository aryan0147/Capstone-Project-2 initialized!

In [2]:
# Set or create an experiment
mlflow.set_experiment("ML-Algos")

<Experiment: artifact_location='mlflow-artifacts:/d43192cc72bb48deb03d07938721755b', creation_time=1739197271560, experiment_id='12', last_update_time=1739197271560, lifecycle_stage='active', name='ML-Algos', tags={}>

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna


In [4]:
df = pd.read_csv('../reddit_preprocessing.csv').dropna()
df.shape

(36662, 2)

In [6]:
import mlflow
import optuna
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 2: TF-IDF vectorization setup
ngram_range = (1, 3)  # Trigram
max_features = 10000  # Max features for TF-IDF
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

# **Step 3: Train-test split BEFORE applying SMOTE (Prevents Data Leakage)**
X_train, X_test, y_train, y_test = train_test_split(
    df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category']
)

# **Step 4: Fit TF-IDF only on training data**
X_train_vec = vectorizer.fit_transform(X_train)  # Fit only on training data
X_test_vec = vectorizer.transform(X_test)  # Transform test data without refitting

# **Step 5: Apply SMOTE on the training set only (Prevents Data Leakage)**
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vec, y_train)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

# Step 6: Optuna objective function for KNN
def objective_knn(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 3, 30)  # Tuning the number of neighbors
    p = trial.suggest_categorical('p', [1, 2])  # Tuning the distance metric (1 for Manhattan, 2 for Euclidean)

    model = KNeighborsClassifier(n_neighbors=n_neighbors, p=p)
    
    # Fit KNN using SMOTE-resampled dataset
    model.fit(X_train_resampled, y_train_resampled)

    # Evaluate accuracy
    y_pred = model.predict(X_test_vec)
    return accuracy_score(y_test, y_pred)

# Step 7: Run Optuna for KNN, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_knn, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'], p=best_params['p'])

    # Log the best model with MLflow
    log_mlflow("KNN", best_model, X_train_resampled, X_test_vec, y_train_resampled, y_test)

# Run the experiment for KNN
run_optuna_experiment()


[I 2025-02-10 20:10:27,889] A new study created in memory with name: no-name-7f5e9937-3617-47e1-9248-e1ddd8f97458
[I 2025-02-10 20:10:31,196] Trial 0 finished with value: 0.3683349243147416 and parameters: {'n_neighbors': 6, 'p': 1}. Best is trial 0 with value: 0.3683349243147416.
[I 2025-02-10 20:10:36,544] Trial 1 finished with value: 0.37542615573435156 and parameters: {'n_neighbors': 13, 'p': 2}. Best is trial 1 with value: 0.37542615573435156.
[I 2025-02-10 20:10:41,754] Trial 2 finished with value: 0.3950634119732715 and parameters: {'n_neighbors': 6, 'p': 2}. Best is trial 2 with value: 0.3950634119732715.
[I 2025-02-10 20:10:45,414] Trial 3 finished with value: 0.3496522569207691 and parameters: {'n_neighbors': 26, 'p': 1}. Best is trial 2 with value: 0.3950634119732715.
[I 2025-02-10 20:10:48,454] Trial 4 finished with value: 0.3578344470203191 and parameters: {'n_neighbors': 13, 'p': 1}. Best is trial 2 with value: 0.3950634119732715.
[I 2025-02-10 20:10:53,744] Trial 5 finis

🏃 View run KNN_SMOTE_TFIDF_Trigrams at: https://dagshub.com/aryan0147/Capstone-Project-2.mlflow/#/experiments/12/runs/e69aca3320094f53898205ece3408688
🧪 View experiment at: https://dagshub.com/aryan0147/Capstone-Project-2.mlflow/#/experiments/12
